<a href="https://colab.research.google.com/github/khal-drog0/AI-ML/blob/learning/DecisionTreeFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# First two columns are features.
# Last column is the class label.

training_data = [
                 ['Green', 3, 'Apple'],
                 ['Yellow', 3, 'Apple'],
                 ['Red', 1, 'Grape'],
                 ['Red', 1, 'Grape'],
                 ['Yellow', 3, 'Lemon']
]

In [18]:
# Column labels
header = ['color', 'diameter', 'label']

In [19]:
# Finding unique values for a column in a dataset
def unique_values(rows, columns):
    return set([row[columns] for row in rows])

In [20]:
# counts the number of each type of example in the dataset
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [21]:
# Test if value is numeric
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [34]:
# Partition the data.
# Record column number and column value.
# Match method compares the feature value of example
# and the feature value stored in the question.
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # Helper method to print the question in readable format
        condition = '=='
        if is_numeric(self.value):
            condition = '>='
        return 'Is %s %s %s?' % (
            header[self.column], condition, str(self.value)
        )

In [23]:
def partition(rows, question):
    # Partition a dataset
    # For each row in the dataset, check if it matches the question. 
    # If so, add it to 'true rows', otherwise, add it to 'false rows'.
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


In [24]:
def gini(rows):
    # Calculate GINI Impurity for al list of rows.
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl ** 2
    return impurity

In [25]:
def info_gain(left, right, current_uncertainty):
    # Information gain
    # Uncertainty of starting node - weighted impurity of two child
    # nodes
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [26]:
def find_best_split(rows):
    # Find best question to ask by iterating over every feature/value
    # and calculating the information gain.
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows]) # unique values in column
        for val in values:
            question = Question(col, val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip hte split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculare the info gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

In [27]:
class Leaf:
    # A Leaf node classifies data.
    # It holds a dictionary of class number of times it appears
    # in the rows form the training data that reach this leaf.
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [28]:
class Decision_Node:
    # A Decision Node asks a question.
    # It holds a reference to the question, and to the two child nodes.
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [29]:
def build_tree(rows):
    # Builds the tree.
    # Uses Recursion
    # Starts by checking for the base case (no further info gain)
    # Prepare for giant stack traces.

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [30]:
def print_tree(node, spacing=''):
    # Base case
    if isinstance(node, Leaf):
        print(spacing +"Predict", node.predictions)
        return
    
    # Print the question at this node
    print(spacing + str(node.question))

    # Call this function recursively on the true branch
    print(spacing +'--> True:')
    print_tree(node.true_branch, spacing +' ')

In [31]:
def classify(row, node):
    # Base case
    if isinstance(node, Leaf):
        return node.predictions
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [32]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + '%'
    return probs

In [35]:
if __name__ == '__main__':
    my_tree = build_tree(training_data)

    print_tree(my_tree)

    testing_data = [
                    ['Green', 3, 'Apple'],
                    ['Yellow', 4, 'Apple'],
                    ['Red', 2, 'Grape'],
                    ['Red', 1, 'Grape'],
                    ['Yellow', 3, 'Lemon']
    ]

    for row in testing_data:
        print("Actual: %s. Predicted: %s" %
              (row[-1], print_leaf(classify(row, my_tree))))

Is diameter >= 3?
--> True:
 Is color == Yellow?
 --> True:
  Predict {'Apple': 1, 'Lemon': 1}
Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
